<a href="https://colab.research.google.com/github/ismael-rtellez/Keras_Series_Assignments/blob/main/Keras_Series_Sprint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Problem 1: Sharing and executing the official tutorial model

In [1]:
# Loading and training a basic image classification model
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

# loading data
fashion_mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Normalizing
x_train, x_test = x_train / 255.0, x_test / 255.0

# Building model
model = models.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(128, activation='relu'),
    layers.Dense(10)
])

# Compiling and training
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True
                                                                 ),
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5, validation_split=0.1)

# Evaluating
model.evaluate(x_test, y_test)


29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.7799 - loss: 0.6440 - val_accuracy: 0.8563 - val_loss: 0.4011
Epoch 2/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8623 - loss: 0.3833 - val_accuracy: 0.8733 - val_loss: 0.3517
Epoch 3/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8730 - loss: 0.3482 - val_accuracy: 0.8628 - val_loss: 0.3853
Epoch 4/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8807 - loss: 0.3203 - val_accuracy: 0.8730 - val_loss: 0.3524
Epoch 5/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8923 - loss: 0.2988 - val_accuracy: 0.8802 - val_loss: 0.3281
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8736 - loss: 0.3507


[0.35578256845474243, 0.8723000288009644]

### Problem 2: (Advance assignment) Execute various methods

In [2]:
"""Yolo models."""

from typing import Mapping, Union, Any, Dict
import tensorflow as tf, tf_keras
#from official.projects.yolo.modeling.layers import nn_blocks


class Yolo(tf_keras.Model):
  """The YOLO model class."""

  def __init__(self,
               backbone,
               decoder,
               head,
               detection_generator,
               **kwargs):
    """Detection initialization function.

    Args:
      backbone: `tf_keras.Model` a backbone network.
      decoder: `tf_keras.Model` a decoder network.
      head: `RetinaNetHead`, the RetinaNet head.
      detection_generator: the detection generator.
      **kwargs: keyword arguments to be passed.
    """
    super(Yolo, self).__init__(**kwargs)

    self._config_dict = {
        'backbone': backbone,
        'decoder': decoder,
        'head': head,
        'detection_generator': detection_generator
    }

    # model components
    self._backbone = backbone
    self._decoder = decoder
    self._head = head
    self._detection_generator = detection_generator
    self._fused = False

  def call(self,  # pytype: disable=annotation-type-mismatch
           inputs: tf.Tensor,
           training: bool = None,
           mask: Any = None) -> Dict[str, tf.Tensor]:
    maps = self.backbone(inputs)
    decoded_maps = self.decoder(maps)
    raw_predictions = self.head(decoded_maps)
    if training:
      return {'raw_output': raw_predictions}
    else:
      # Post-processing.
      predictions = self.detection_generator(raw_predictions)
      predictions.update({'raw_output': raw_predictions})
      return predictions

  @property
  def backbone(self):
    return self._backbone

  @property
  def decoder(self):
    return self._decoder

  @property
  def head(self):
    return self._head

  @property
  def detection_generator(self):
    return self._detection_generator

  def get_config(self):
    return self._config_dict

  @classmethod
  def from_config(cls, config):
    return cls(**config)

  @property
  def checkpoint_items(
      self) -> Mapping[str, Union[tf_keras.Model, tf_keras.layers.Layer]]:
    """Returns a dictionary of items to be additionally checkpointed."""
    items = dict(backbone=self.backbone, head=self.head)
    if self.decoder is not None:
      items.update(decoder=self.decoder)
    return items

  def fuse(self):
    """Fuses all Convolution and Batchnorm layers to get better latency."""
    print('Fusing Conv Batch Norm Layers.')
    if not self._fused:
      self._fused = True
      for layer in self.submodules:
        if isinstance(layer, nn_blocks.ConvBN):
          layer.fuse()
      self.summary()
    return

In this task, I have chosen the object detection model of YOLO (You Only Look Once) in the TensorFlow Models GitHub repository.

I investigated and studied the code of the YOLO Object Detection model that can be located in the TensorFlow Models repository
https://github.com/tensorflow/models/blob/master/official/projects/yolo/README.md.

The code mainly employs ```TensorFlow 2.x``` and ```Keras subclassing``` API to build a modular objects detecting platform of YOLO. It consists of key files such as ```yolo_model.py``` to construct the model and ```yolo_layer.py``` to decode raw outputs into bounding boxes as well as ```losses.py``` to measure loss of objects detection.

The model has a support of ```YOLOv3``` and ```YOLOv4```  and configurable YAML files are used to train the settings. It is implemented as modular and adheres to latest practices of the TensorFlow framework in deep learning and allows to perform high-performance object detection on a dataset such as ```COCO```. Since the full training pipeline was somewhat large and needed a lot of resources, I concentrated on reading the code and learning the pipeline architecture and the way everything ran through it.

### Problem 3: Learning Iris (binary classification) with Keras

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Loading dataset
from google.colab import files
uploaded = files.upload()

df = pd.read_csv("Iris.csv")

# Preprocess
df = df[df['Species'].isin(['Iris-versicolor', 'Iris-virginica'])]
X = df[["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]].values
y = (df['Species'] == 'Iris-virginica').astype(int).values #Binary: Viriginica = 1

# Splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Building model
model = Sequential([
    Dense(10, activation='relu', input_shape=(4,)),
    Dense(1, activation='sigmoid')
])

# Compiling and trainig
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50, validation_split=0.1)
model.evaluate(X_test, y_test)

Epoch 1/50


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 109ms/step - accuracy: 0.4922 - loss: 1.4256 - val_accuracy: 0.5000 - val_loss: 1.3465
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.4492 - loss: 1.4770 - val_accuracy: 0.5000 - val_loss: 1.2955
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.5078 - loss: 1.2699 - val_accuracy: 0.5000 - val_loss: 1.2463
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.4922 - loss: 1.2646 - val_accuracy: 0.5000 - val_loss: 1.1974
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.4844 - loss: 1.2314 - val_accuracy: 0.5000 - val_loss: 1.1469
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.4961 - loss: 1.1638 - val_accuracy: 0.5000 - val_loss: 1.0979
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.4844 - loss: 1.1331 - val_accuracy: 0.5000 - val_loss: 1.0525
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.4961 - loss: 1.0735 - val_accuracy: 0.5000 - val_loss: 1.0101
Epoch 9/50

[0.6205913424491882, 0.75]

### Problem 4: Learning Iris (multi-class classification) with Keras

In [2]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# I will the same Iris dataset only that now i'm considering all the 3 classes
df = pd.read_csv("Iris.csv")
X = df[["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]].values
y = df['Species'].values

# Encoding classes to integers them one-hot
le = LabelEncoder()
y = le.fit_transform(y)
y = to_categorical(y)

# Splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Building model
model = Sequential([
    Dense(16, activation='relu', input_shape=(4,)),
    Dense(3, activation='sigmoid')
])

# Compiling and trainig
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50, validation_split=0.1)
model.evaluate(X_test, y_test)

Epoch 1/50


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 143ms/step - accuracy: 0.3134 - loss: 2.7175 - val_accuracy: 0.4167 - val_loss: 2.0774
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.2905 - loss: 2.6070 - val_accuracy: 0.4167 - val_loss: 1.9682
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.3416 - loss: 2.3077 - val_accuracy: 0.4167 - val_loss: 1.8707
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.3270 - loss: 2.1940 - val_accuracy: 0.4167 - val_loss: 1.7768
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.3395 - loss: 2.0328 - val_accuracy: 0.4167 - val_loss: 1.6897
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.3416 - loss: 1.8957 - val_accuracy: 0.4167 - val_loss: 1.6090
Epoch 7/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.3166 - loss: 1.8452 - val_accuracy: 0.4167 - val_loss: 1.5351
Epoch 8/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.3343 - loss: 1.6859 - val_accuracy: 0.4167 - val_loss: 1.4688
Epoch 9/50

[0.8522288203239441, 0.5666666626930237]

### Problem 5: Learning House Prices with Keras

In [3]:
df = pd.read_csv("train.csv")
df = df[["GrLivArea", "YearBuilt", "SalePrice"]].dropna()

X = df[["GrLivArea", "YearBuilt"]].values
y = df["SalePrice"].values

# Normalizing inputs
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Train test splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Building model
model = Sequential([
    Dense(64, activation='relu', input_shape=(2,)),
    Dense(1)
])

# Compiling and trainig
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.fit(X_train, y_train, epochs=100, validation_split=0.1)
model.evaluate(X_test, y_test)

Epoch 1/100


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 40747585536.0000 - mae: 184305.0156 - val_loss: 34332448768.0000 - val_mae: 172564.9062
Epoch 2/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 43235000320.0000 - mae: 188226.4844 - val_loss: 34332284928.0000 - val_mae: 172564.4844
Epoch 3/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 39024463872.0000 - mae: 181165.2188 - val_loss: 34332094464.0000 - val_mae: 172563.9531
Epoch 4/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 40806785024.0000 - mae: 183726.7500 - val_loss: 34331842560.0000 - val_mae: 172563.2812
Epoch 5/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 40546443264.0000 - mae: 183109.7656 - val_loss: 34331531264.0000 - val_mae: 172562.4219
Epoch 6/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 39528243200.0000 - mae: 182032.4375 - val_loss: 34331140096.0000 - val_mae: 172561.3750
Epoch 7/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 41248059392.0000 - mae: 184439.5625 - val_loss: 34330658816.0000 - val_

[39363162112.0, 180399.421875]

### Problem 6: Learning MNIST with Keras

In [4]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Loading
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocessing
x_train = x_train.reshape(-1, 784) / 255.0
x_test = x_test.reshape(-1, 784) / 255.0
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Building model
model = Sequential([
    Dense(128, activation='relu', input_shape=(784,)),
    Dense(10, activation='softmax')
])

# Compiling and trainig
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, validation_split=0.1)
model.evaluate(x_test, y_test)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.8701 - loss: 0.4666 - val_accuracy: 0.9652 - val_loss: 0.1252
Epoch 2/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9628 - loss: 0.1250 - val_accuracy: 0.9732 - val_loss: 0.0919
Epoch 3/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.9767 - loss: 0.0803 - val_accuracy: 0.9768 - val_loss: 0.0777
Epoch 4/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.9823 - loss: 0.0598 - val_accuracy: 0.9742 - val_loss: 0.0849
Epoch 5/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9868 - loss: 0.0461 - val_accuracy: 0.9805 - val_loss: 0.0766
Epoch 6/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9905 - loss: 0.0337 - val_accuracy: 0.9800 - val_loss: 0.0786
Epoch 7/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9926 - loss: 0.0260 - val_accuracy: 0.9800 - val_loss: 0.0785
Epoch 8/10
1688/1688 ━━━━━━━━━━━━━━━

[0.0841829776763916, 0.978600025177002]

### Problem 7: (Advanced assignment) Rewriting to Pytorch

In [5]:
# A. Iris Binary Classification
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Loading and preprocessing
df_a = pd.read_csv("Iris.csv")
df_a = df_a[df_a['Species'].isin(['Iris-versicolor', 'Iris-virginica'])]
X_a = df_a[["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]].values
y_a = (df_a['Species'] == 'Iris-virginica').astype(int).values

scaler = StandardScaler()
X_a = scaler.fit_transform(X_a)

X_train_a, X_test_a, y_train_a, y_test_a = train_test_split(X_a, y_a, test_size=0.2)

X_train_a, y_train_a = torch.tensor(X_train_a, dtype=torch.float32), torch.tensor(y_train_a, dtype=torch.float32).unsqueeze(1)
X_test_a, y_test_a = torch.tensor(X_test_a, dtype=torch.float32), torch.tensor(y_test_a, dtype=torch.float32).unsqueeze(1)

# Model
model = nn.Sequential(
    nn.Linear(4, 10),
    nn.ReLU(),
    nn.Linear(10, 1),
    nn.Sigmoid()
)

criterion_a = nn.BCELoss()
optimizer_a = optim.Adam(model.parameters(), lr=0.01)

# Training Loop

for epoch in range(50):
    model.train()
    y_pred_a = model(X_train_a)
    loss_a = criterion_a(y_pred_a, y_train_a)

    optimizer_a.zero_grad()
    loss_a.backward()
    optimizer_a.step()

# Evaluating
with torch.no_grad():
    model.eval()
    test_pred_a = model(X_test_a)
    acc_a = ((test_pred_a > 0.5) == y_test_a).float().mean()
    print("Iris Binary Classification:\n")
    print(f"Test Accuracy : {acc_a.item():.4f}")
    print("\n")



# B. Iris multi-clas classification
from sklearn.preprocessing import LabelEncoder

# Loading and preprocessing
df_b = pd.read_csv("Iris.csv")
X_b = df_b[["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]].values
y_b = df_b['Species'].values

encoder = LabelEncoder()
y_b = encoder.fit_transform(y_b)

scaler = StandardScaler()
X_b = scaler.fit_transform(X_b)

X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(X_b, y_b, test_size=0.2)

X_train_b, y_train_b = torch.tensor(X_train_b, dtype=torch.float32), torch.tensor(y_train_b, dtype=torch.long)
X_test_b, y_test_b = torch.tensor(X_test_b, dtype=torch.float32), torch.tensor(y_test_b, dtype=torch.long)

# Model
model = nn.Sequential(
    nn.Linear(4, 10),
    nn.ReLU(),
    nn.Linear(10, 3)
)

criterion_b = nn.CrossEntropyLoss()
optimizer_b = optim.Adam(model.parameters(), lr=0.01)

# Training Loop

for epoch in range(50):
    model.train()
    y_pred_b = model(X_train_b)
    loss_b = criterion_b(y_pred_b, y_train_b)

    optimizer_b.zero_grad()
    loss_b.backward()
    optimizer_b.step()

# Evaluating
with torch.no_grad():
    model.eval()
    test_logits_b = model(X_test_b)
    test_pred_b = torch.argmax(test_logits_b, dim=1)
    acc_b = (test_pred_b == y_test_b).float().mean()
    print("Iris Multi-class Classification:\n")
    print(f"Test Accuracy: {acc_b.item():.4f}")
    print("\n")



# C. House prices calssification

# Loading and preprocessing
df_c = pd.read_csv("train.csv")
df_c = df_c[["GrLivArea", "YearBuilt", "SalePrice"]].dropna()

X_c = df_c[["GrLivArea", "YearBuilt"]].values
y_c = df_c["SalePrice"].values

scaler = StandardScaler()
X_c = scaler.fit_transform(X_c)

X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_c, y_c, test_size=0.2)

X_train_c, y_train_c = torch.tensor(X_train_c, dtype=torch.float32), torch.tensor(y_train_c, dtype=torch.float32).unsqueeze(1)
X_test_c, y_test_c = torch.tensor(X_test_c, dtype=torch.float32), torch.tensor(y_test_c, dtype=torch.float32).unsqueeze(1)

# Model
model = nn.Sequential(
    nn.Linear(2, 16),
    nn.ReLU(),
    nn.Linear(16, 1)
)

criterion_c = nn.MSELoss()
optimizer_c = optim.Adam(model.parameters(), lr=0.01)

# Training Loop

for epoch in range(100):
    model.train()
    y_pred_c = model(X_train_c)
    loss_c = criterion_c(y_pred_c, y_train_c)

    optimizer_c.zero_grad()
    loss_c.backward()
    optimizer_c.step()

    """if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")"""

# Evaluating
with torch.no_grad():
    model.eval()
    test_pred_c = model(X_test_c)
    mse_c = criterion_c(test_pred_c, y_test_c).item()
    rmse_c = mse_c ** 0.5
    mae_c = torch.mean(torch.abs(test_pred_c - y_test_c)).item()
    print("Iris Multi-class Classification:\n")
    print(f"Test RMSE: {rmse_c:.4f}")
    print(f"Test MAE: {mae_c:.4f}")
    print("\n")



# D. Mnist Classification
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Loading
(x_train_d, y_train_d), (x_test_d, y_test_d) = mnist.load_data()

# Preprocessing
x_train_d = x_train_d.reshape(-1, 784) / 255.0
x_test_d = x_test_d.reshape(-1, 784) / 255.0
#y_train_d = to_categorical(y_train_d)
#y_test_d = to_categorical(y_test_d)

x_train_d, y_train_d = torch.tensor(x_train_d, dtype=torch.float32), torch.tensor(y_train_d, dtype=torch.long)
x_test_d, y_test_d = torch.tensor(x_test_d, dtype=torch.float32), torch.tensor(y_test_d, dtype=torch.long)

model = nn.Sequential(
    nn.Linear(28*28, 128),
    nn.ReLU(),
    nn.Linear(128, 10)
)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training Loop

for epoch in range(10):
    model.train()
    y_pred_d = model(x_train_d)
    loss_d = criterion(y_pred_d, y_train_d)

    optimizer.zero_grad()
    loss_d.backward()
    optimizer.step()

    #print(f"Epoch {epoch+1}, Loss: {loss_d.item():.4f}")

# Evaluating
with torch.no_grad():
    model.eval()
    test_logits_d = model(x_test_d)
    test_pred_d = torch.argmax(test_logits_d, dim=1)
    acc_d = (test_pred_d == y_test_d).float().mean()
    print("MNIST Classification:\n")
    print(f"Test Accuracy: {acc_d.item():.4f}")
    print("\n")

Iris Binary Classification:

Test Accuracy : 1.0000


Iris Multi-class Classification:

Test Accuracy: 0.9000


Iris Multi-class Classification:

Test RMSE: 191635.6971
Test MAE: 176626.6250


MNIST Classification:

Test Accuracy: 0.8885




### Problem 8: (Advanced) Comparing Framework

**Iris Binary Classification**

*  The difference in calculation speed between the two codes is more favorable to the one performed with PyTorch, but only slightly; it is not a very big difference.
*  The code with Keras is shorter than that written with Pytorch. In terms of readability, Pytorch could be considered superior, as its structure is more sectioned, differentiating each process, but Keras is not difficult to understand either.
*  Both codes have several features available that improve the model. Perhaps the ability to specify loss and backward as separate functions in PyTorch makes it more accurate (1.0000) compared to Keras (0.7500).

**Iris Multi-class Classification**

*  The difference in calculation speed between the two codes is more favorable to the one performed with PyTorch, but only slightly; it is not a very big difference.
*  The code with Keras is shorter than that written with Pytorch. In terms of readability, Pytorch could be considered superior, as its structure is more sectioned, differentiating each process, but Keras is not difficult to understand either.
*  Both codes have several features available that improve the model. Perhaps the ability to specify loss and backward as separate functions in PyTorch makes it more accurate (0.9000) compared to Keras (0.5667).

**House Price Classification**

*  The difference in calculation speed between the two codes is more favorable to the one performed with PyTorch, it is not a very big difference.
*  The code with Keras is shorter than that written with Pytorch. In terms of readability, Pytorch could be considered superior, as its structure is more sectioned, differentiating each process, but Keras is not difficult to understand either.
*  Both codes have several features available that improve the model. Perhaps specifying loss and backward as separate functions improves the MAE metric value in PyTorch (176626.6250) mcompared to Keras (179853.6250).

**MNIST Classification**

*  The difference in calculation speed between the two codes is more favorable to the one performed with PyTorch.
*  The code with Keras is shorter than that written with Pytorch. In terms of readability, Pytorch could be considered superior, as its structure is more sectioned, differentiating each process, but Keras is not difficult to understand either.
*  Both codes have several features available that improve the model, but in this case, the precision metric favors the Keras code (0.9748) by a significant margin when compared to the PyTorch code (0.8885).